In [1]:
import os
import datetime
import warnings
import polars as pl
import pandas as pd
import altair as alt

from src.najdi_rok import najdi_rok
from src.pocet_stran import pocet_stran
from src.bez_bordelu import bez_bordelu
from src.alt_friendly import alt_friendly
from src.hezke_jmeno import hezke_jmeno
from src.kristi_promin import kristi_promin
from src.zjisti_vazbu import zjisti_vazbu

pl.Config(tbl_rows=100)
alt.data_transformers.disable_max_rows()
alt.themes.register('irozhlas', kristi_promin)
alt.themes.enable('irozhlas')
warnings.filterwarnings('ignore')

In [2]:
df = pl.read_parquet(os.path.join("data/cnb_sloupce","100.parquet"))
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","leader.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","008.parquet")), left_on="001", right_on="001", how="left")
df = df.to_pandas()
df = df[df["leader"].str[6].isin(["a", "t"])]
df = df[~df["leader"].str[7].isin(["b", "i", "s", " "])]
df = df[(df["008"].str[15:17] == "xr") & (df["008"].str[35:38] == "cze")]
df = pl.from_pandas(df)
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","020.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","022.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","041.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","245.parquet")), left_on="001", right_on="001", how="left")
df = df.join(pl.read_parquet(os.path.join("data/cnb_sloupce","300.parquet")), left_on="001", right_on="001", how="left")
df = df.explode("022_a").filter(pl.col("022_a").is_null())
df = df.with_columns(pl.col('008').map_elements(najdi_rok, return_dtype=int).alias('rok'))
df = df.with_columns(pl.col('300_a').map_elements(pocet_stran, return_dtype=int).alias('stran'))
df = df.with_columns(pl.col('245_a').map_elements(bez_bordelu, return_dtype=str))
df = df.explode("020_q").with_columns(pl.col("020_q").map_elements(zjisti_vazbu, return_dtype=str).alias('vazba'))
df = df.explode('245_p').with_columns(pl.col('245_p').map_elements(bez_bordelu, return_dtype=str))
print(len(df))
df = df.sort(by="rok")
df = df.filter((~pl.col("245_h").str.contains("grafika")) | pl.col("245_h").is_null()).unique(subset=["100_a","245_a"], keep="first")
print(len(df))

795736
566175


In [38]:
df = df.filter(pl.col("rok") >= 1800)

In [3]:
preklady = df.explode("041_h").drop_nulls(subset=["041_h","rok"]).filter(pl.col("041_h") != "cze")

In [4]:
preklady_celkem = preklady.group_by(['rok']).len().rename({'len':'prekladu_celkem'})
preklady_celkem

rok,prekladu_celkem
i64,u32
1977,615
1980,547
1837,12
1840,31
1846,18
1971,458
1843,26
1989,410
1849,24


In [5]:
nahradit = {
    'eng': 'angličtina', 'ger': 'němčina', 'slo': 'slovenština', 'rus': 'ruština', 'fre': 'francouzština', 'lat': 'latina', 'pol': 'polština','ita':'italština'
}

In [6]:
nejzastoupenejsi_jazyky = preklady.group_by('041_h').len().sort('len', descending=True)
nejzastoupenejsi_jazyky = nejzastoupenejsi_jazyky.get_column('041_h').to_list()
nejzastoupenejsi_jazyky = nejzastoupenejsi_jazyky[0:7]
print(nejzastoupenejsi_jazyky)
nejzastoupenejsi_jazyky = ['eng','ger','rus','fre','slo']
preklady_po_jazycich = preklady.filter(pl.col("rok").is_between(1900,2020)).group_by(['rok','041_h']).len().rename({'len':'prekladu'})
preklady_po_jazycich = preklady_po_jazycich.join(preklady_celkem, on='rok', how='full')
preklady_po_jazycich = preklady_po_jazycich.with_columns((pl.col('prekladu') / pl.col('prekladu_celkem')).alias('podíl'))
preklady_po_jazycich = preklady_po_jazycich.with_columns(pl.col('041_h').cast(str))
preklady_po_jazycich = preklady_po_jazycich.filter(pl.col("041_h").is_in(nejzastoupenejsi_jazyky)).sort('rok').rename({'041_h':'jazyk'}).with_columns(pl.col("rok").map_elements(lambda x: datetime.date(year=int(x), month=1, day=1), return_dtype=pl.Date).cast(pl.Datetime))
preklady_po_jazycich = preklady_po_jazycich.with_columns(pl.col('jazyk').replace_strict(nahradit))
preklady_po_jazycich

['eng', 'ger', 'rus', 'fre', 'pol', 'ita', 'slo']


rok,jazyk,prekladu,rok_right,prekladu_celkem,podíl
datetime[μs],str,u32,i64,u32,f64
1900-01-01 00:00:00,"""slovenština""",1,1900,178,0.005618
1900-01-01 00:00:00,"""angličtina""",41,1900,178,0.230337
1900-01-01 00:00:00,"""francouzština""",42,1900,178,0.235955
1900-01-01 00:00:00,"""němčina""",29,1900,178,0.162921
1900-01-01 00:00:00,"""ruština""",17,1900,178,0.095506
1901-01-01 00:00:00,"""němčina""",23,1901,211,0.109005
1901-01-01 00:00:00,"""ruština""",21,1901,211,0.099526
1901-01-01 00:00:00,"""slovenština""",1,1901,211,0.004739
1901-01-01 00:00:00,"""angličtina""",34,1901,211,0.161137


In [7]:
titulek = "Ze kterých jazyků se kdy překládaly knihy do češtiny"
podtitulek = ["Podíl jazyků na překladové literatuře – krásné, naučné i odborné."] #,"Angličtina a francouzština (téměř) zmizely jen během 2. světové války,","kdy dominovaly překlady z němčiny. Po osvobození raketově narostl podíl ruštiny."]
kredit = "zdroj dat: Česká národní bibliografie · vizualizace: iROZHLAS.cz · 2025"

In [8]:
podily_faceted = alt.Chart(preklady_po_jazycich.to_pandas(), title={'text': titulek}, width=300).mark_area().encode(
    alt.X("rok:T", 
          title=None,
          axis=alt.Axis(domainOpacity=0, tickColor='#DCDDD6')
    ),
    alt.Y('podíl:Q', 
          axis=alt.Axis(labelExpr="datum.label * 100 + ' %'", orient='right', domainOpacity=0, tickColor='#DCDDD6'), 
          title=None
    ),
    alt.Color("jazyk:N", 
              scale=alt.Scale(range=['#687fa9','#84c0e4','#e3d83b','#b45058','#789256','#E0DAB5',]), ## '# ['#a7ddd6','#001f80','#e6e6e4','#e5db00','#e49996','#9cc002']), 
              title=None, 
              legend=None
    ),
    row=alt.Row(
        "jazyk:N", 
        title=None, 
        spacing=15, 
        header=alt.Header(labelAngle=0, labelAlign='left', labelAnchor='middle', labelFont='Asap')
    )
).resolve_axis(
    x='independent'
).properties(height=60, width=300).configure_view(stroke='transparent')

podily_faceted

alt.Chart(...)

## Počty jednotlivých jazyků

In [46]:
rocni_pocty = preklady.filter(pl.col("rok") >= 1800).group_by("rok").agg(pl.col("041_h").unique().len()).sort(by="rok")
rocni_pocty

rok,041_h
i64,u32
1801,2
1802,2
1803,1
1804,3
1805,2
1806,2
1807,3
1808,2
1809,2


In [48]:
alt_friendly(rocni_pocty)

,rok,041_h
0,1801-01-01,2
1,1802-01-01,2
2,1803-01-01,1
3,1804-01-01,3
4,1805-01-01,2
...,...,...
220,2021-01-01,46
221,2022-01-01,46
222,2023-01-01,47
223,2024-01-01,48


In [70]:
alt.Chart(alt_friendly(rocni_pocty), width=300,
         title=["Z kolika jednotlivých jazyků se překládaly knihy"]
         ).mark_area().encode(
    alt.X("rok:T", axis=alt.Axis(domainOpacity=0, tickCount=4, tickColor='#DCDDD6'), title=None),
    alt.Y("041_h", axis=alt.Axis(orient='right', tickCount=4, domainOpacity=0, tickColor='#DCDDD6'), title=None)
)

alt.Chart(...)